In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime as dt

import numpy as np

from deep_hedging import Underlyings, Ticker, ConstantRateCurve
from deep_hedging.non_linear import (
    WorstOfCall,
    WorstOfPut,
    WorstOfCallTwoAssets,
    WorstOfPutTwoAssets,
)
from deep_hedging.monte_carlo import GBMPricer

RANDOM_SEED = 12

In [3]:
N_PATHS = 100_000
N_STOCKS = 2
TILL_MATURITY = 3.0
N_DAYS = 365 * TILL_MATURITY
RF_RATE = 0.03
VOL1, VOL2 = 0.2, 0.3
CORR = 0.4

In [4]:
N_DAYS

1095.0

In [5]:
np.random.seed(RANDOM_SEED)

start = dt.datetime(2019, 1, 1)
end = start + dt.timedelta(days=N_DAYS)

var_covar = np.array([[VOL1**2, CORR * VOL1 * VOL2], [CORR * VOL1 * VOL2, VOL2**2]])

underlyings = Underlyings(
    tickers=[Ticker(f"Stock {i + 1}", f"{i + 1}") for i in range(N_STOCKS)],
    start=start,
    end=end,
    dividends=np.array([0.0] * N_STOCKS),
    means=np.array([RF_RATE] * N_STOCKS),
    var_covar=var_covar,
)
underlyings.data

In [6]:
np.sqrt(np.diag(underlyings.get_var_covar()))

array([0.2, 0.3])

In [7]:
var_covar

array([[0.04 , 0.024],
       [0.024, 0.09 ]])

In [8]:
curve = ConstantRateCurve(constant_rate=RF_RATE)

## Exchange Option.

In [9]:
worst_of_call = WorstOfCallTwoAssets(
    underlyings=underlyings,
    yield_curve=curve,
    strike_level=0.0,
    start_date=start,
    end_date=end,
)

In [10]:
pricer = GBMPricer(payoff_function=worst_of_call.payoff, random_seed=RANDOM_SEED)

In [11]:
worst_of_call.yield_curve.to_present_value(pricer.future_value(
    spot=[1.0, 1.0],
    time_till_maturity=TILL_MATURITY,
    risk_free_rate_fn=curve.get_instant_fwd_rate,
    dividends_fn=lambda t: 0.0,
    var_covar_fn=lambda t: np.array([var_covar] * len(t)),
    n_paths=N_PATHS,
), worst_of_call.days_till_maturity)

0.8035274260802041

In [12]:
worst_of_call.price()

0.804140646599248

In [13]:
worst_of_call._closed_out_price(spot_start=[1.0, 1.0])

0.8041406465983341

## Worst-Of Call.

In [14]:
worst_of_call = WorstOfCallTwoAssets(
    underlyings=underlyings,
    yield_curve=curve,
    strike_level=1.0,
    start_date=start,
    end_date=end,
)

In [15]:
pricer = GBMPricer(payoff_function=worst_of_call.payoff, random_seed=RANDOM_SEED)
worst_of_call.yield_curve.to_present_value(pricer.future_value(
    spot=[1.0, 1.0],
    time_till_maturity=TILL_MATURITY,
    risk_free_rate_fn=curve.get_instant_fwd_rate,
    dividends_fn=lambda t: 0.0,
    var_covar_fn=lambda t: np.array([var_covar] * len(t)),
    n_paths=N_PATHS,
), worst_of_call.days_till_maturity)

0.08348976798244326

In [16]:
worst_of_call.price()

0.08383147759977133

## Worst-Of Put.

In [17]:
worst_of_put = WorstOfPutTwoAssets(
    underlyings=underlyings,
    yield_curve=curve,
    strike_level=1.0,
    start_date=start,
    end_date=end,
)

In [18]:
pricer = GBMPricer(payoff_function=worst_of_put.payoff, random_seed=RANDOM_SEED)
worst_of_call.yield_curve.to_present_value(pricer.future_value(
    spot=[1.0, 1.0],
    time_till_maturity=TILL_MATURITY,
    risk_free_rate_fn=curve.get_instant_fwd_rate,
    dividends_fn=lambda t: 0.0,
    var_covar_fn=lambda t: np.array([var_covar] * len(t)),
    n_paths=N_PATHS,
), worst_of_call.days_till_maturity)

0.19389352717346717

In [19]:
worst_of_put.price()

0.19362201627175152